# Traveler's Dilemma
Simple Game

*Ghi chú:* **Các giải thuật dưới đây là những lời giải tốt nhất tìm được để chứng minh cho kết luận của bài toán. Các giải thuật khác đã thử nghiệm và kết quả của chúng được ghi trong báo cáo.**

## Resource
Add the necessary packages and declare some useful functions.

In [1]:
import Pkg

function addPackage(pkg::String)
    if !haskey(Pkg.installed(), pkg)
        Pkg.add(pkg)
    end
end

addPackage("Distributions")
addPackage("LinearAlgebra")
addPackage("JuMP")
addPackage("Ipopt")

using Distributions, LinearAlgebra, JuMP, Ipopt, Random


# Appendices (Algorithms for Decision Making)
# G.5 Convenience Functions
struct SetCategorical{S}
    elements::Vector{S} # Set elements (could be repeated)
    distr::Categorical # Categorical distribution over set elements

    # Trả về SetCategorical với phân phối đều
    function SetCategorical(elements::AbstractVector{S}) where {S}
        weights = ones(length(elements))
        return new{S}(elements, Categorical(normalize(weights, 1)))
    end

    # Trả về SetCategorical với phân phối weights
    function SetCategorical(elements::AbstractVector{S}, weights::AbstractVector{Float64}) where {S}
        ℓ₁ = norm(weights, 1)
        if ℓ₁ < 1e-6 || isinf(ℓ₁)
            return SetCategorical(elements)
        end
        distr = Categorical(normalize(weights, 1))
        return new{S}(elements, distr)
    end
end

# Trả về phần tử ngẫu nhiên trong SetCategorical D
Distributions.rand(D::SetCategorical) = D.elements[rand(D.distr)]
Distributions.rand(D::SetCategorical, n::Int) = D.elements[rand(D.distr, n)]

function Distributions.pdf(D::SetCategorical, x)
    sum(e == x ? w : 0.0 for (e, w) in zip(D.elements, D.distr.p))
end


┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\Pkg\src\Pkg.jl:595
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\Pkg\src\Pkg.jl:595
    Updating registry at `C:\Users\Admin\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Admin\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Admin\.julia\environments\v1.7\Manifest.toml`
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\Pkg\src\Pkg.jl:595
┌ Warning: Pkg.installed() is deprecated
└ @ Pkg C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\Pkg\src\Pkg.jl:595


## Define simple game structures 

In [2]:
# Algorithm 24.1 (Algorithms for Decision Making)
struct SimpleGame
    γ # discount factor
    ℐ # agents
    𝒜 # joint action space
    R # joint reward function
end


# Algorithm 24.2 (Algorithms for Decision Making)
struct SimpleGamePolicy
    p # dictionary mapping actions to probabilities

    # Trả về policy ngẫu nhiên
    function SimpleGamePolicy(p::Base.Generator)
        return SimpleGamePolicy(Dict(p))
    end

    # Trả về policy từ dict
    function SimpleGamePolicy(p::Dict)
        vs = collect(values(p))
        vs ./= sum(vs)
        return new(Dict(k => v for (k, v) in zip(keys(p), vs)))
    end

    SimpleGamePolicy(ai) = new(Dict(ai => 1.0))
end

# Trả về xác suất của ai
(πi::SimpleGamePolicy)(ai) = get(πi.p, ai, 0.0)

# Trả về action ngẫu nhiên trong policy πi
function (πi::SimpleGamePolicy)()
    D = SetCategorical(collect(keys(πi.p)), collect(values(πi.p)))
    return rand(D)
end

# Trả về tích các vector
joint(X) = vec(collect(Iterators.product(X...)))

joint(π, πi, i) = [i == j ? πi : πj for (j, πj) in enumerate(π)] # helper of best_response

# Hàm lợi ích kỳ vọng từ 𝒫 và policy π của agent i
function utility(𝒫::SimpleGame, π, i)
    𝒜, R = 𝒫.𝒜, 𝒫.R
    p(a) = prod(πj(aj) for (πj, aj) in zip(π, a))
    return sum(R(a)[i] * p(a) for a in joint(𝒜))
end


# Algorithm 24.3 (Algorithms for Decision Making)
# Trả về deterministic policy tốt nhất
function best_response(𝒫::SimpleGame, π, i)
    U(ai) = utility(𝒫, joint(π, SimpleGamePolicy(ai), i), i)
    ai = argmax(U, 𝒫.𝒜[i])
    return SimpleGamePolicy(ai)
end

best_response (generic function with 1 method)

## Problem properties
Register the properties of Traveler's Dilemma problem.

In [3]:
# Khai báo số agent và action của mỗi agent
const N_AGENTS = 2
ACTIONS = vec(collect(2:100))

# Hàm joint reward
function joint_reward(a::Tuple{Int64,Int64})
    ai, aj = a
    return ai == aj ? (ai, aj) : (ai < aj ? (ai + 2, ai - 2) : (aj - 2, aj + 2))
end

# Biến trò chơi của bài toán
travelersDilemma = SimpleGame(
    1.0,
    vec(collect(1:N_AGENTS)),
    [ACTIONS for _ in 1:N_AGENTS],
    joint_reward)

SimpleGame(1.0, [1, 2], [[2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]], joint_reward)

## Hierarchical Softmax solution

In [4]:
# Algorithm 24.4 (Algorithms for Decision Making)
# Hàm trả về policy của mô hình softmax
function softmax_response(𝒫::SimpleGame, π, i, λ)
    𝒜i = 𝒫.𝒜[i]
    U(ai) = utility(𝒫, joint(π, SimpleGamePolicy(ai), i), i)
    return SimpleGamePolicy(ai => exp(λ * U(ai)) for ai in 𝒜i)
end


# Algorithm 24.9 (Algorithms for Decision Making)
# Cấu trúc mô tả thông số của mô hình Hierarchical Softmax
struct HierarchicalSoftmax
    λ # precision parameter
    k # level
    π # initial policy
end

# Constructor của HierarchicalSoftmax
function HierarchicalSoftmax(𝒫::SimpleGame, λ, k)
    π = [SimpleGamePolicy(ai => 1.0 for ai in 𝒜i) for 𝒜i in 𝒫.𝒜]
    return HierarchicalSoftmax(λ, k, π)
end

# Hàm lặp để điều chỉnh mô hình softmax
function solve(M::HierarchicalSoftmax, 𝒫)
    π = M.π
    for k in 1:M.k
        π = [softmax_response(𝒫, π, i, M.λ) for i in 𝒫.ℐ]
    end
    return π
end

# Giải
π = solve(HierarchicalSoftmax(travelersDilemma, 0.3, 4), travelersDilemma)

π¹ = π[1].p
π² = π[2].p

# Ghi kết quả
for a in ACTIONS
    println(a => (π¹[a], π²[a]))
end

2 => (3.3216244433691576e-13, 3.3216244433691576e-13)
3 => (4.4837240103375595e-13, 4.4837240103375595e-13)
4 => (6.052394346085835e-13, 6.052394346085835e-13)
5 => (8.169877814970728e-13, 8.169877814970728e-13)
6 => (1.102818152532633e-12, 1.102818152532633e-12)
7 => (1.4886487963448598e-12, 1.4886487963448598e-12)
8 => (2.0094656891222545e-12, 2.0094656891222545e-12)
9 => (2.7124949589624926e-12, 2.7124949589624926e-12)
10 => (3.661485210822247e-12, 3.661485210822247e-12)
11 => (4.9424880605662504e-12, 4.9424880605662504e-12)
12 => (6.671661039763351e-12, 6.671661039763351e-12)
13 => (9.005800415445694e-12, 9.005800415445694e-12)
14 => (1.2156559009620783e-11, 1.2156559009620783e-11)
15 => (1.640963824814931e-11, 1.640963824814931e-11)
16 => (2.2150694716930613e-11, 2.2150694716930613e-11)
17 => (2.990031035489309e-11, 2.990031035489309e-11)
18 => (4.0361197276913615e-11, 4.0361197276913615e-11)
19 => (5.4481917619626686e-11, 5.4481917619626686e-11)
20 => (7.354289633620451e-11, 7.35

## Reduce the number of actions available
Reduce the number of actions available from \\$100 to \\$20 maximum value can be put down for saving runtime. It is easy to see that this does not greatly affect the conclusion of the problem.

In [5]:
# Giảm số action của mỗi agent xuống 19
ACTIONS = vec(collect(2:20))

travelersDilemma = SimpleGame(
    1.0,
    vec(collect(1:N_AGENTS)),
    [ACTIONS for _ in 1:N_AGENTS],
    joint_reward)

SimpleGame(1.0, [1, 2], [[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]], joint_reward)

## Correlated Equilibrium solution
General of Nash equilibrium concept by relaxing the assumption that the agents act independently.

In [6]:
# Algorithm 24.6 (Algorithms for Decision Making)
# Policy của Correlated Equilibrium
mutable struct JointCorrelatedPolicy
    p # dictionary mapping from joint actions to probabilities
    JointCorrelatedPolicy(p::Base.Generator) = new(Dict(p))
end

# Trả về policy của joint action a
(π::JointCorrelatedPolicy)(a) = get(π.p, a, 0.0)

# Trả về action ngẫu nhiên trong policy π
function (π::JointCorrelatedPolicy)()
    D = SetCategorical(collect(keys(π.p)), collect(values(π.p)))
    return rand(D)
end


# Algorithm 24.7 (Algorithms for Decision Making) Utilitarian | Fixed bug | Modified
struct CorrelatedEquilibrium end

# Hàm đổi action trong công thức
joint(a, ai′, i) = Tuple(k == i ? ai′ : v for (k, v) in enumerate(a))

function solve(M::CorrelatedEquilibrium, 𝒫::SimpleGame)
    ℐ, 𝒜, R = 𝒫.ℐ, 𝒫.𝒜, 𝒫.R
    model = Model(Ipopt.Optimizer)
    # Khai báo cũng như ràng buộc 3
    @variable(model, π[joint(𝒜)] ≥ 0)
    # Hàm mục tiêu
    @objective(model, Max, sum(sum(π[a] * R(a)[i] for a in joint(𝒜)) for i in ℐ))
    # Ràng buộc 1
    @constraint(model, [i = ℐ, ai = 𝒜[i], ai′ = 𝒜[i]],
        sum(R(a)[i] * π[a] for a in joint(𝒜) if a[i] == ai)
        ≥
        sum(R(joint(a, ai′, i))[i] * π[a] for a in joint(𝒜) if a[i] == ai))
    # Ràng buộc 2
    @constraint(model, sum(π) == 1)
    # Tối ưu mô hình
    optimize!(model)
    return JointCorrelatedPolicy(a => value(π[a]) for a in joint(𝒜))
end


# Giải
π = solve(CorrelatedEquilibrium(), travelersDilemma)

# Ghi kết quả
for x in π.p
    println(x)
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      361
Number of nonzeros in inequality constraint Jacobian.:     8632
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      361
                     variables with only lower bounds:      361
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

(18, 5) => 2.087526051978087e-7
(3, 17) => 1.4965573539306171e-7
(14, 8) => 2.0839210210315207e-7
(6, 3) => 3.1253796919086625e-6
(18, 11) => 4.28019383543445e-8
(14, 18) => 3.2495349597761206e-8
(16, 5) => 2.6918692533745513e-7
(16, 11) => 4.936211563709422e-8
(6, 20) => 1.4792068547618404e-7
(19, 5) => 2.70415139169565e-8
(11, 9) => 4.374072331225218e-7
(15, 10) => 4.653250244828489e-8
(19, 11) => 7.959706200542695e-9
(15, 2) => 2.0501742995617286e-7
(4, 3) => 0.0
(13, 3) => 6.522198943360241e-7
(8, 7) => 0.0
(4, 20) => 2.4092157873022967e-7
(13, 20) => 2.2496344139771652e-8
(2, 9) => 1.2696108668048826e-6
(10, 9) => 0.0
(14, 7) => 2.280911626535458e-7
(6, 10) => 1.0059439978808668e-6
(6, 2) => 3.908630399571853e-6
(11, 3) => 1.2183587587458364e-6
(20, 9) => 6.888571927097305e-8
(5, 6) => 0.0
(11, 20) => 4.061574087123665e-8
(17, 5) => 9.108986625313171e-8
(7, 4) => 1.804642000228517e-6
(17, 11) => 2.4987159662275327e-8
(9, 9) => 1.274245084997635e-6
(4, 10) => 1.453523211354797e-6
(

## Fictitious Play solution

In [7]:
# Algorithm 24.11 (Algorithms for Decision Making)
# Mô hình Fictitious Play
mutable struct FictitiousPlay
    𝒫 # simple game
    i # agent index
    N # array of action count dictionaries
    πi # current policy
end

# Constructor của mô hình
function FictitiousPlay(𝒫::SimpleGame, i)
    N = [Dict(aj => 1 for aj in 𝒫.𝒜[j]) for j in 𝒫.ℐ]
    πi = SimpleGamePolicy(ai => 1.0 for ai in 𝒫.𝒜[i])
    return FictitiousPlay(𝒫, i, N, πi)
end

# Trả về action ngẫu nhiên từ policy πi
(πi::FictitiousPlay)() = πi.πi()

# Trả về xác suất của action ai
(πi::FictitiousPlay)(ai) = πi.πi(ai)

# Cập nhật mô hình Fictitious Play
function update!(πi::FictitiousPlay, a)
    N, 𝒫, ℐ, i = πi.N, πi.𝒫, πi.𝒫.ℐ, πi.i
    # Tăng số lần các action xuất hiện
    for (j, aj) in enumerate(a)
        N[j][aj] += 1
    end
    # Cập nhật policy mới
    p(j) = SimpleGamePolicy(aj => u / sum(values(N[j])) for (aj, u) in N[j])
    π = [p(j) for j in ℐ]
    πi.πi = best_response(𝒫, π, i)
end


# Algorithm 24.10 (Algorithms for Decision Making)
# Lặp để cập nhật mô hình
function simulate(𝒫::SimpleGame, π, k_max)
    for k = 1:k_max
        a = [πi() for πi in π]
        for πi in π
            update!(πi, a)
        end
    end
    return π
end

# Thử nghiệm các tham số k_max và ghi kết quả
for k_max in [100, 1000, 10000, 100000]
    π = simulate(
        travelersDilemma,
        [FictitiousPlay(travelersDilemma, i) for i in travelersDilemma.ℐ],
        k_max)

    println("After ", k_max, " iterations, the (deterministic) policy:")
    
    π¹ = π[1].πi
    π² = π[2].πi
    
    println("π¹ = ", π¹)
    println("π² = ", π²)
    println()
end

After 100 iterations, the (deterministic) policy:
π¹ = SimpleGamePolicy(Dict(11 => 1.0))
π² = SimpleGamePolicy(Dict(11 => 1.0))

After 1000 iterations, the (deterministic) policy:
π¹ = SimpleGamePolicy(Dict(8 => 1.0))
π² = SimpleGamePolicy(Dict(8 => 1.0))

After 10000 iterations, the (deterministic) policy:
π¹ = SimpleGamePolicy(Dict(4 => 1.0))
π² = SimpleGamePolicy(Dict(4 => 1.0))

After 100000 iterations, the (deterministic) policy:
π¹ = SimpleGamePolicy(Dict(2 => 1.0))
π² = SimpleGamePolicy(Dict(2 => 1.0))

